In [1]:
import os
import gc
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

In [2]:
import cudf
import pandas as pd
import xgboost as xgb
from sklearn import datasets
cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)
pd.set_option('display.max_columns', 500)

In [4]:
features = [
            'user_user_count',
            'user_item_count',
            'remember_ratio',
            'price',
            'price_power_0_mean',
            'price_power_1_mean',
            'price_power_2_mean',
            'price_power_3_mean',
            'price_power_5_mean',
            'nn_logits',
            'jp',
            'de',
            'uk',
           ]
bad = [

       72,73,74,75,76,77,#11 string
       101,102,103,104,105,#15 brand
       108,109,110,111,112,#16 author
       115,116,117,118,119,#17 material
       ]
feature_list =  [[200,206]]
for i in [1,2,3,4,5,6,7,9,10,11,12,13,16,15,17,18,19,21,22,23,27]:
    feature_list.append([(i-1)*7+1,i*7])
for i in feature_list:
    for j in range(i[0],i[1]+1):
        
        if j in bad:
            continue
        features+=['ui_feat'+str(j)]

In [5]:
valid = []
data_set = ['JP']
for locale__2 in data_set :#['DE','JP','UK']
    temp_valid = pd.read_parquet('valid5_data_'+locale__2+'_0~0.pqt')
    a = [0]*len(temp_valid)
    b = [0]*len(temp_valid)
    c = [0]*len(temp_valid)
    if locale__2 == 'DE':
        a = [1]*len(temp_valid)
    if locale__2 == 'JP':
        b = [1]*len(temp_valid)
    if locale__2 == 'UK':
        c = [1]*len(temp_valid)
    temp_valid['de']=a
    temp_valid['jp']=b
    temp_valid['uk']=c
    valid.append(temp_valid)
    
valid = pd.concat(valid).reset_index(drop=True)
for row in tqdm(feature_list):
    temp_df = []
    for locale__2 in data_set :#['DE','JP','UK']
        temp_df.append(pd.read_parquet('valid5_data_'+locale__2+'_'+str(row[0])+'~'+str(row[1])+'.pqt'))
    temp_df = pd.concat(temp_df).reset_index(drop=True)
    
    for j in range(row[0],row[1]+1):
        if j in bad:
            continue
        valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
    del temp_df

X_valid = valid[features]
y_valid = valid['target']
dvalid = xgb.DMatrix(cudf.DataFrame(X_valid),cudf.DataFrame( y_valid), group=[250] * (len(X_valid)//250) ) 

 65%|██████▌   | 15/23 [00:01<00:00,  9.82it/s]/tmp/ipykernel_1996358/583505980.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996358/583505980.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996358/583505980.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [6]:
len(features)

153

# GPU

In [7]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, batch_size=1024*256):
        self.features = features
        self.target = 'target'
        self.df = df
        self.it = 0 # set iterator to 0
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        dt = cudf.DataFrame(self.df.iloc[a:b])
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        self.it += 1
        return 1

# mrr100

In [8]:
import numpy as np
from typing import Tuple
def mrr_at_k(predt: np.ndarray, dtrain: xgb.DMatrix, k: int) -> Tuple[str, float]:
    y = dtrain.get_label()
    group_sizes = dtrain.get_group()
    assert len(y) == sum(group_sizes), "group sizes must match data length"
    assert len(predt) == sum(group_sizes), "group sizes must match prediction length"
    mrr_sum = 0.
    mrr_count = 0
    start_idx = 0
    for i, group_size in enumerate(group_sizes):
        end_idx = start_idx + group_size
        group_y = y[start_idx:end_idx]
        group_predt = predt[start_idx:end_idx]
        order = np.argsort(-group_predt)
        ranks = np.zeros_like(order)
        ranks[order] = np.arange(len(group_predt))
        reciprocal_ranks = 1. / (ranks + 1.)
        if (group_y > 0).sum() == 0:
            mrr = 0.
        else:
            mrr = np.mean(reciprocal_ranks[group_y > 0])
        mrr_sum += mrr
        mrr_count += 1
        start_idx = end_idx
    mrr_at_k = mrr_sum / mrr_count if mrr_count > 0 else 0.
    return f"mrr@{k}", mrr_at_k

# train

In [9]:
train = []
data_set = ['DE','JP','UK']
for locale__2 in data_set :
    temp_train = pd.read_parquet('train5_data_'+locale__2+'_0~0.pqt')
    a = [0]*len(temp_train)
    b = [0]*len(temp_train)
    c = [0]*len(temp_train)
    if locale__2 == 'DE':
        a = [1]*len(temp_train)
    if locale__2 == 'JP':
        b = [1]*len(temp_train)
    if locale__2 == 'UK':
        c = [1]*len(temp_train)
    temp_train['de']=a
    temp_train['jp']=b
    temp_train['uk']=c
    train.append(temp_train)
    
train = pd.concat(train).reset_index(drop=True)
for row in tqdm(feature_list):
    temp_df = []
    for locale__2 in data_set :
        temp_df.append(pd.read_parquet('train5_data_'+locale__2+'_'+str(row[0])+'~'+str(row[1])+'.pqt'))
    temp_df = pd.concat(temp_df).reset_index(drop=True)
    
    for j in range(row[0],row[1]+1):
        if j in bad:
            continue
        train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
    del temp_df

Xy_train = IterLoadForDMatrix(train, features)
dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)
dtrain.set_group([250] * (len(train)//250))

 65%|██████▌   | 15/23 [02:08<01:25, 10.63s/it]/tmp/ipykernel_1996358/1071215783.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996358/1071215783.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
 70%|██████▉   | 16/23 [02:21<01:20, 11.44s/it]/tmp/ipykernel_1996358/1071215783.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

In [10]:
target = valid['target'].tolist()
rank = valid['rank'].tolist()
mrr = 0
for i in range(len(rank)//250):
    for j in range(100):
        if target[i*250+j] == 1:
            mrr+=1/(1+j)
mrr/len(rank)*250
#0.3539636726293081

0.3512649190064116

In [11]:

for hp in [1]:
    print("############################################")
    print('max_depth: ',hp)
    xgb_parms = { 
            'max_depth':4, 
            'learning_rate':0.07, 
            'subsample':0.5,
            'colsample_bytree':0.6,
            'reg_lambda':0.05,
            'eval_metric':'map',
            'objective':'binary:logistic',
            # 'objective':'rank:pairwise',
            'scale_pos_weight':8,
            'tree_method':'gpu_hist',
            'predictor':'gpu_predictor',
            'num_feature':len(features),
            'random_state':42 
        }
    model = None

    for epoch in range(1):
        for i in range(0,1):
            
            _ = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                feval=lambda predt, dtrain: mrr_at_k(predt, dtrain, k=100),
                num_boost_round=1,
                early_stopping_rounds=0,
                verbose_eval=1,
                xgb_model=model)
            model = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=3500,
                early_stopping_rounds=200,
                verbose_eval=100,
                xgb_model=model)
            _ = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                feval=lambda predt, dtrain: mrr_at_k(predt, dtrain, k=100),
                num_boost_round=1,
                early_stopping_rounds=0,
                verbose_eval=1,
                xgb_model=model)#0.36880

############################################
max_depth:  1


/home/sjliu/anaconda3/envs/rapids-23.02/lib/python3.10/site-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-map:0.59339	train-mrr@100:0.31731	valid-map:0.61166	valid-mrr@100:0.34936
[0]	train-map:0.59339	valid-map:0.61166
[100]	train-map:0.63055	valid-map:0.64886
[200]	train-map:0.63928	valid-map:0.66077
[300]	train-map:0.64153	valid-map:0.66348
[400]	train-map:0.64300	valid-map:0.66349
[500]	train-map:0.64391	valid-map:0.66444
[600]	train-map:0.64483	valid-map:0.66471
[700]	train-map:0.64533	valid-map:0.66451
[800]	train-map:0.64588	valid-map:0.66436
[900]	train-map:0.64640	valid-map:0.66504
[1000]	train-map:0.64683	valid-map:0.66524
[1100]	train-map:0.64725	valid-map:0.66460
[1136]	train-map:0.64741	valid-map:0.66459
[0]	train-map:0.64741	train-mrr@100:0.37666	valid-map:0.66455	valid-mrr@100:0.40811


In [11]:
a = features#list(model.get_score(importance_type='gain').keys())
b = list(model.get_score(importance_type='gain').values())
Z = zip(b, a)  # 对AB进行封装，把频率放在前面
Z = sorted(Z, reverse=True)  # 进行逆序排列
b, a = zip(*Z)  # 进行解压，其中的AB已经按照频率排好
for i,j in zip(b, a):
    print(i,'\t',j)

15085.6591796875 	 ui_feat187
14723.5029296875 	 ui_feat161
8814.6640625 	 ui_feat6
8428.3837890625 	 nn_logits
6728.07470703125 	 ui_feat185
4011.457275390625 	 ui_feat67
3109.309326171875 	 ui_feat188
3011.5244140625 	 ui_feat90
2917.0830078125 	 ui_feat70
2397.842529296875 	 ui_feat66
2325.269775390625 	 ui_feat65
2251.427734375 	 ui_feat91
2100.471435546875 	 ui_feat123
2005.0399169921875 	 ui_feat88
1900.083251953125 	 ui_feat48
1760.7314453125 	 ui_feat68
1518.6558837890625 	 ui_feat121
1478.08056640625 	 ui_feat7
1240.62060546875 	 ui_feat125
1041.3966064453125 	 ui_feat84
1034.052001953125 	 ui_feat85
1028.663818359375 	 ui_feat122
1022.7891845703125 	 ui_feat143
864.6069946289062 	 ui_feat186
660.3601684570312 	 ui_feat133
653.9398803710938 	 ui_feat36
643.6629638671875 	 ui_feat41
623.8285522460938 	 ui_feat69
615.6431884765625 	 ui_feat42
597.433837890625 	 ui_feat46
558.5595703125 	 ui_feat200
526.03759765625 	 ui_feat99
524.6970825195312 	 ui_feat146
518.7783813476562 	 ui

In [ ]:
import os
 
pid = os.getpid()
!kill -9 $pid

# test

In [5]:
# # Test
xgb_parms = { 
            'max_depth':4, 
            'learning_rate':0.07, 
            'subsample':0.5,
            'colsample_bytree':0.6,
            'reg_lambda':0.05,
            'eval_metric':'map',
            'objective':'binary:logistic',
            'scale_pos_weight':8,
            'tree_method':'gpu_hist',
            'predictor':'gpu_predictor',
            'random_state':42
        }
model = xgb.Booster()


In [30]:
recall1 = []
le = 0
for locale__ in tqdm(['DE','JP','UK']):

    model = xgb.Booster()
    model.load_model(f'XGB_locale_sum3.xgb')
    data = cudf.read_parquet('test5_data_'+locale__+'_0~0.pqt')
    a = [0]*len(data)
    b = [0]*len(data)
    c = [0]*len(data)
    if locale__ == 'DE':
        a = [1]*len(data)
    if locale__ == 'JP':
        b = [1]*len(data)
    if locale__ == 'UK':
        c = [1]*len(data)
    data['de']=cudf.DataFrame(a)
    data['jp']=cudf.DataFrame(b)
    data['uk']=cudf.DataFrame(c)
    if len(data)==0:
        break
    for row in feature_list:
        if row[0]==200:
            temp_df = cudf.read_parquet('test5_data_'+locale__+'_'+str(row[0])+'~'+str(row[1])+'.pqt')
        else:
            temp_df = cudf.read_parquet('test5_data_'+locale__+'_'+str(row[0])+'~'+str(row[1])+'.pqt')
        for j in range(row[0],row[1]+1):
            if j in bad:
                continue
            data['ui_feat'+str(j)] = cudf.DataFrame(temp_df['ui_feat'+str(j)])
    le+=len(data)

    dtest = xgb.DMatrix(data=cudf.DataFrame(data[features]))
    preds = model.predict(dtest)
    preds = np.array(preds)*np.log(np.e+10*data['sum_pop_score'].to_pandas())
    del dtest
    predictions = cudf.DataFrame(data[['user', 'item']])
    predictions['pred'] = preds
    predictions = predictions.sort_values(['user', 'pred'], ascending=[True, False]).reset_index(drop=True)
    predictions['rk'] = predictions.groupby('user').item.cumcount().astype('int16')
    predictions = predictions.loc[predictions.rk < 150]
    predictions = predictions.to_pandas()
    predictions = predictions.groupby('user').item.apply(list).to_frame().reset_index()
    predictions.columns = ['test_user', 'next_item_prediction']
    recall1 += predictions['next_item_prediction'].tolist()
    del predictions

100%|██████████| 3/3 [03:51<00:00, 77.22s/it]


### 直接生成

In [34]:
import pickle
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
locale = pd.read_pickle('../data/test_data_p2.dataset').reset_index(drop=True)['locale']
result = [[product_id2asin[x] for x in row[:100]] for row in recall1]
result = pd.DataFrame({'locale':locale,'next_item_prediction':result})
result.to_parquet( 'submission_locale.parquet', engine='pyarrow')
result

,locale,next_item_prediction
0,DE,"[B091CK241X, B07SDFLVKD, B0BGC82WVW, B093X59B3..."
1,DE,"[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B004P4OF1..."
2,DE,"[B09Z4PZQBF, B08LLF9M11, B09GPJ15GS, B09KBCTXF..."
3,DE,"[B07Y1KLF25, B07QQZD49D, B07T5XJW9G, B07SZHKGZ..."
4,DE,"[B0B2JY9THB, B08SXLWXH9, B08YK8FQJ8, B08P94RML..."
...,...,...
316967,UK,"[B07GKP2LCF, B07GKYSHB4, B006DDGCI2, B00V6FIFZ..."
316968,UK,"[B00M35Y326, B08NPR9NFX, B086HRFHQH, B085C7TCT..."
316969,UK,"[B08VDHH6QF, B08VD5DC5L, B07QK2SPP7, B08VDGZ8K..."
316970,UK,"[B089CZWB4C, B09WCQYGX8, B08W2JJZBM, B089CJ7P7..."


In [21]:
import pickle
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
locale = pd.read_pickle('../data/test_data_p2.dataset').reset_index(drop=True)['locale']
result = [[product_id2asin[x] for x in row[:100]] for row in recall1]
result = pd.DataFrame({'locale':locale,'next_item_prediction':result})
result.to_parquet( 'submission_locale.parquet', engine='pyarrow')
result

,locale,next_item_prediction
0,DE,"[B091CK241X, B07SDFLVKD, B0BGC82WVW, B093X59B3..."
1,DE,"[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B004P4OF1..."
2,DE,"[B09Z4PZQBF, B08LLF9M11, B09GPJ15GS, B09KBCTXF..."
3,DE,"[B07Y1KLF25, B07QQZD49D, B07T5XJW9G, B07SZHKGZ..."
4,DE,"[B0B2JY9THB, B08SXLWXH9, B08YK8FQJ8, B08P94RML..."
...,...,...
316967,UK,"[B07GKP2LCF, B07GKYSHB4, B006DDGCI2, B00V6FIFZ..."
316968,UK,"[B00M35Y326, B08NPR9NFX, B086HRFHQH, B085C7TCT..."
316969,UK,"[B08VDHH6QF, B08VD5DC5L, B07QK2SPP7, B08VDGZ8K..."
316970,UK,"[B089CZWB4C, B09WCQYGX8, B08W2JJZBM, B089CJ7P7..."


In [19]:
import pickle
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
locale = pd.read_pickle('../data/test_data_p2.dataset').reset_index(drop=True)['locale']
result = [[product_id2asin[x] for x in row[:100]] for row in recall1]
result = pd.DataFrame({'locale':locale,'next_item_prediction':result})
result.to_parquet( 'submission_locale_sum.parquet', engine='pyarrow')
result

,locale,next_item_prediction
0,DE,"[B091CK241X, B07SDFLVKD, B0BGC82WVW, B093X59B3..."
1,DE,"[B084CB7GX9, B08HQWQ1SK, B004P4QFJM, B004P4OF1..."
2,DE,"[B09Z4PZQBF, B08LLF9M11, B09GPJ15GS, B09KBCTXF..."
3,DE,"[B07Y1KLF25, B07QQZD49D, B07T5XJW9G, B07SZHKGZ..."
4,DE,"[B0B2JY9THB, B08SXLWXH9, B08YK8FQJ8, B08P94RML..."
...,...,...
316967,UK,"[B07GKP2LCF, B07GKYSHB4, B006DDGCI2, B016RAAUE..."
316968,UK,"[B00M35Y326, B08NPR9NFX, B086HRFHQH, B085C7TCT..."
316969,UK,"[B08VDHH6QF, B08VD5DC5L, B07QK2SPP7, B08VDGZ8K..."
316970,UK,"[B089CZWB4C, B09WCQYGX8, B08W2JJZBM, B089CJ7P7..."


,locale,next_item_prediction
0,DE,"[B091CK241X, B07SDFLVKD, B0BGC82WVW, B093X59B3..."
1,DE,"[B084CB7GX9, B004P4QFJM, B08HQWQ1SK, B004P4OF1..."
2,DE,"[B09Z4PZQBF, B08LLF9M11, B09GPJ15GS, B09KBCTXF..."
3,DE,"[B07Y1KLF25, B07QQZD49D, B07T5XJW9G, B07SZHKGZ..."
4,DE,"[B0B2JY9THB, B08SXLWXH9, B08YK8FQJ8, B08P94RML..."
...,...,...
316967,UK,"[B07GKP2LCF, B07GKYSHB4, B006DDGCI2, B00V6FIFZ..."
316968,UK,"[B00M35Y326, B08NPR9NFX, B086HRFHQH, B085C7TCT..."
316969,UK,"[B08VDHH6QF, B08VD5DC5L, B07QK2SPP7, B08VDGZ8K..."
316970,UK,"[B089CZWB4C, B09WCQYGX8, B08W2JJZBM, B089CJ7P7..."


### 联合

In [8]:
b = pd.read_parquet( 'submission_locale_sum.parquet')

In [9]:
a = pd.read_parquet( '../task1/submission_locale_sum.parquet')

In [11]:
recall1 = b['next_item_prediction'].tolist()
recall2 = a['next_item_prediction'].tolist()

In [14]:
recall = []
for i in tqdm(range(len(recall1))):
    output = []
    for j in range(100):
        if str(recall1[i][j]) not in output:
            output.append(recall1[i][j])
        if str(recall2[i][j]) not in output:
            output.append(recall2[i][j])
    recall.append(output[:100])
a['next_item_prediction'] = recall
a.to_parquet( 'submission_locale_xgb.parquet', engine='pyarrow')

100%|██████████| 316972/316972 [01:22<00:00, 3856.43it/s]
